In [1]:
#Add libraries
import pandas as pd, requests
import concurrent.futures
import os
import time
import numpy as np

In [2]:
#Add credentials
f = open("credentials2.txt", "r") 
cred=f.read().split(',') 
PAIRS_SERVER = cred[0] 
PAIRS_USER = cred[1]
PAIRS_CREDENTIALS=tuple([PAIRS_USER,cred[2]])
guiURL=cred[4]
APIkey=cred[5]
tokenurl='https://auth-b2b-twc.ibm.com/auth/GetBearerForClient/'+cred[5]+'/ibm-pairs'
#accessToken = 'Bearer '+requests.get(tokenurl).json()['access_token']
accessToken = requests.get(tokenurl).json()['access_token']

In [3]:
#Datasets to download
T_id=49423     #Ground temperature (ERA5)
u10e_id=49420  #10m wind speed East  (ERA5)
u10n_id=49421  #10m wind speed North (ERA5)
TD_id=49422    #Relative humidity (ERA5)
GP_id=49439    #Ground pressure (ERA5)
Rcs_id=49461   #Clear sky (ERA5)
Rs_id=49440    #Solar raditiaton (ERA5)

In [4]:
#Creates a list of query_jsons to be submitted
items=os.listdir()
df=pd.read_csv('GIS/TMCF.csv')
df

,longitude,latitude,id,tmcf,country,realm
0,-155.589264,-14.966681,0,Ulla Ulla,BOL,Neotropic
1,-105.816653,-17.750014,1,Amboró,BOL,Neotropic
2,-105.000014,-17.266653,2,Carrasco,BOL,Neotropic
3,-104.166653,-20.000014,3,Tariquía,BOL,Neotropic
4,-103.616681,-22.383319,4,Itatiaia,BRA,Neotropic
...,...,...,...,...,...,...
524,149.600014,2.199681,524,Hose Mountains,MYS,Indomalayan
525,150.283042,2.322625,525,Linau Balui Plateau,MYS,Indomalayan
526,150.549597,2.896486,526,Penambo Range,MYS,Indomalayan
527,152.057653,3.306375,527,Dulit Range,MYS,Indomalayan


In [5]:
query_jsons=[]
for i in range(0, len(df)):
    lon=df['longitude'].iloc[i]
    lat=df['latitude'].iloc[i]

    query_json = {
         "layers" : [
             {"type" : "raster", "id" : T_id},
             {"type" : "raster", "id" : u10e_id},
             {"type" : "raster", "id" : u10n_id},
             {"type" : "raster", "id" : TD_id},
             {"type" : "raster", "id" : GP_id},
             {"type" : "raster", "id" : Rcs_id},
             {"type" : "raster", "id" : Rs_id},
          ],
          "name":i,
          "spatial" : {"type" : "point",  "coordinates" : [lat, lon]},
          "temporal" : {"intervals" : [{"start" : "1997-01-01T00:00:00Z", "end" : "2020-12-31T00:00:00Z"}]}
     }
    
    if not 'ts'+str(i)+'.csv' in items: 
        query_jsons.append(query_json)

print(len(query_jsons)) 

3


In [6]:
#Function to submit a query
def submitQuery(query_json):
    api_response = requests.post('https://pairs.res.ibm.com/v2/query',headers = {'Authorization':'Bearer {}'.format(accessToken),'Content-Type': 'application/json'},json = query_json)
    pairs_data = pd.DataFrame(api_response.json()['data'])
    pairs_data['datetime'] = pd.to_datetime(pairs_data['timestamp'] * 1e6, errors = 'coerce')
    pairs_data.drop(['layerName','dataset','timestamp','longitude','latitude'], axis=1, inplace=True)
    name=query_json.get('name')
    pairs_data.to_csv('ts'+str(name)+'.csv')

    return api_response, name

In [7]:
#Submit and donwload querys
CONNECTIONS=10

start=time.perf_counter()

with concurrent.futures.ThreadPoolExecutor(max_workers=CONNECTIONS) as executor:
    results=[executor.submit(submitQuery,query_json) for query_json in query_jsons]    
    for f in concurrent.futures.as_completed(results):
        print(f.result())

finish=time.perf_counter()

print('Finished in:', round(finish-start,2), ' seconds')

(<Response [200]>, 346)
(<Response [200]>, 454)
(<Response [200]>, 345)
Finished in: 217.38  seconds
